Virtual Machines: Using Docker, Ansible and EC2
=================================================

<img src="logo_docker.png" />
<img src="ansible.jpg" /> 

# Table of Contents
1. Introduction to Configuration Management
  - Containers
  - Configuration Management
  - Docker
    - what is Docker
    - Origin and History
  - Ansible
    - what is Docker
    - Origin and History

Development and Deployment with Docker and Ansible
===========================

What problem does Docker solve?
-------------------------------
Docker allows you as a developer or system administrator to run a contained and consistent environment on your own machine, containing one or many nodes, with low overhead.

<b>Docker uses resource isolation features of the Linux kernel such as [cgroups](https://en.wikipedia.org/wiki/Cgroups) and kernel [namespaces](http://man7.org/linux/man-pages/man7/namespaces.7.html) to allow independent "containers" to run within a single Linux instance, avoiding the overhead of starting virtual machines.</b>

Local Development with Docker
------------------------------

What problem does Ansible solve?
------------------------------
Ansible is an open source configuration management tool, directly targeted at solving the problem of having to manually configure each machine by logging in with `ssh` and installing software and configuring.

Key aspects of Ansible:  
- only needs to be installed on 1 machine, which we might call ‘Ansible Management Host’
- configures machines by ssh-ing into them and running commands
- easily add machines to your infrastructure
- idempotence: an operation has the same result whether run once or many times. In code, you define a desired state such as 'state=present' or 'state=running'


- Ansible has many built-in modules that facilitate the provisioning of EC2 instances ('ec2' module), or the setup of Docker containers ('docker' module).

How to use Ansible:
- have Ansible Management Node, either your laptop or some server. this is where we install Ansible software
two config files: Host inventory file, this has list of machines (either IP addresses or host names) you want to provision. Can be grouped together. (note: this can also be a database if you have e.g. 1000s of hosts)
- Playbooks are files that describe configuration tasks that need to be performed against hosts given by hosts file
- Plays: the goal of a Play in a Playbook is to map a group of systems into multiple roles
- Modules: Ansible ships with a ‘module library’ that contains modules that can be executed directly on remote hosts. these modules can control system resources, or handle executing system commands. modules return JSON format data.
can also run in ad-hoc mode, against a set of nodes

workflow:
- you run an Ansible playbook on the management node, providing playbook as command line argument. 
Ansible looks through the playbook and notices we’ve targeted a certain set of nodes, e.g. the ‘web’ group
- it reads in inventory file and finds hosts under the appropriate group
- it logs into those mahines with ssh (note: you’ll want to establish a key so you don’t have to provide password)
- finally, you get status report

Contrasting Ansible with tools like Puppet and Chef:
- Ansible pushed config out to machines via ssh; so Ansible only needs to be installed on the Ansible mgt machine, and each host it ssh’s into only needs Python and SSH installed

## Using Docker: Introduction

(Note: in the lab, you will install Docker and Ansible on your own machine.)

### Launching your first Docker Container

On the command line, execute the following:

```bash
$ sudo docker run -i -t ubuntu /bin/bash
```

When you execute the 'run' command, Docker will:
- look for the image name, indicated by the -t flag, locally and download the image if it is not found. So the first time you run this command you do not have the image locally, and Docker will automatically download the 'ubuntu' image from the Docker image registry.
- run a new container based on the ubuntu image
- launch the bash process within the container

After executing this command, you should see something similar to the following; notice how the first line is in bash on your local machine, and the last line is in bash within the Ubunutu container.

```yaml
bash-3.2$ docker run -i -t ubuntu /bin/bash
Unable to find image 'ubuntu:latest' locally
latest: Pulling from ubuntu

6071b4945dcf: Pull complete 
5bff21ba5409: Pull complete 
e5855facec0b: Pull complete 
8251da35e7a7: Already exists 
ubuntu:latest: The image you are pulling has been verified. Important: image verification is a tech preview feature and should not be relied on to provide security.

Digest: sha256:946dd32914a9b315d45e1983b75a597eb931cc59c179402d4fbc7d1e1dd8129d
Status: Downloaded newer image for ubuntu:latest
root@f0122ebb41a7:/# 
```

There is almost nothing running on your container, as we can illustrate with the ifconfig and ps commands:

```yaml
root@f0122ebb41a7:/# ifconfig
eth0      Link encap:Ethernet  HWaddr 02:42:ac:11:00:34  
          inet addr:172.17.0.52  Bcast:0.0.0.0  Mask:255.255.0.0
          inet6 addr: fe80::42:acff:fe11:34/64 Scope:Link
          UP BROADCAST RUNNING MULTICAST  MTU:1500  Metric:1
          RX packets:7 errors:0 dropped:2 overruns:0 frame:0
          TX packets:9 errors:0 dropped:0 overruns:0 carrier:0
          collisions:0 txqueuelen:0 
          RX bytes:558 (558.0 B)  TX bytes:738 (738.0 B)

lo        Link encap:Local Loopback  
          inet addr:127.0.0.1  Mask:255.0.0.0
          inet6 addr: ::1/128 Scope:Host
          UP LOOPBACK RUNNING  MTU:65536  Metric:1
          RX packets:0 errors:0 dropped:0 overruns:0 frame:0
          TX packets:0 errors:0 dropped:0 overruns:0 carrier:0
          collisions:0 txqueuelen:0 
          RX bytes:0 (0.0 B)  TX bytes:0 (0.0 B)

root@f0122ebb41a7:/# ps
  PID TTY          TIME CMD
    1 ?        00:00:00 bash
   15 ?        00:00:00 ps

```

We can exit our container by simply typing 'exit':

```yaml
root@f0122ebb41a7:/# exit
exit
bash-3.2$
```

We can view a list of Docker containers that are present on your machine:

```yaml
bash-3.2$ docker images
REPOSITORY          TAG                 IMAGE ID            CREATED             VIRTUAL SIZE
ubuntu              latest              8251da35e7a7        2 days ago          188.4 MB
bash-3.2$
```

Let's try to 'run' our container again and create a file in the root directory:

```yaml
bash-3.2$ docker run -i -t ubuntu /bin/bash
root@8e8167f9c206:/# ls
bin  boot  dev  etc  home  lib  lib64  media  mnt  opt  proc  root  run  sbin  srv  sys  tmp  usr  var
root@8e8167f9c206:/# touch myfile.txt
root@8e8167f9c206:/# ls
bin  boot  dev  etc  home  lib  lib64  media  mnt  myfile.txt  opt  proc  root  run  sbin  srv  sys  tmp  usr  var
root@8e8167f9c206:/# exit
exit
bash-3.2$ docker run -i -t ubuntu /bin/bash
root@ea6c66d47902:/# ls
bin  boot  dev  etc  home  lib  lib64  media  mnt  opt  proc  root  run  sbin  srv  sys  tmp  usr  var
root@ea6c66d47902:/#
```

Notice:
- The second and third time you ran the container, it started much faster than the first time. This is because once an image has been downloaded, it is very fast to launch a container based on that image. <B>This speed is one of Docker's strengths.</b>
- Changes you make to Docker containers are <b>ephemeral</b>: when you exit the container, <b>those changes disappear</b>. We created an empty file myfile.txt inside the container, then exited and ran the container again. The file had disappeared.
  - there are ways to make changes to containers <b>persist</b>, which we won't go into right now.

An empty container is not that useful, so we'll want to fill it with some things. We might want to have it fulfil some purpose -- when we start using Ansible, this will be described as a <b>role</b>.

We can add things to our Container by creating a <b>Dockerfile</b>: a plain-text file that instructs Docker step-by-step how to build a Docker <b>image</b>, which we an then use to run a Docker <b>container</b>. 

Here is an example of a simple Dockerfile:

```bash
FROM ubuntu:12.04
MAINTAINER Galvanize Student "gstudent@galvanize.com"
RUN echo "deb http://archive.ubuntu.com/ubuntu precise main universe" > /etc/apt/sources.list
RUN apt-get update
RUN apt-get install -y nginx
EXPOSE 80
CMD ["nginx"]
```

How does Ansible work?
=============
<img src="ansible-modus-operandi.png">

### Hosts file
Here is an example of a hosts file:

```yaml
--- 
[loadbalancer]
balance01
balance02

[hdfs]
storage1
storage2
storage3

[messageservice]
kafka01
192.168.0.1
192.168.0.2
```

Key points:
- notice how the file starts with triple dashes "---" which is necessary to indicate YAML syntax
- the file contains 3 groups, each between brackets: [].
- host names can be indicated either by a hostname or IP address.

### Playbooks
#### A Note on YAML Syntax
Ansible playbooks are written in the YAML data serialization format. Some basics about YAML syntax: 
The first line of every Ansible file must consist of “---” for it to be interpreted correctly by.
Items that begin with a - are considered list items. 
Items that have the format of key: value operate as hashes or dictionaries.

```yaml
---
- hosts: webservers
  remote_user: root

  tasks:
  - name: Test Connection
    ping:

  - name: make sure apache is running
    service: name=httpd state=running
```

#### Playbooks
The added code above does two things: 
- adds tasks to be executed on the set of hosts we defined before
- adds another set of hosts, ‘databases’, with its own set of tasks to be executed against it
- adds a task to install a piece of software: (code that was added)

Let's run this playbook and see what happens:

```bash
$ ansible-playbook playbook.yml
```

```bash
$ [running playbook]
```

As your infrastructure becomes more sophisticated, and you add more machines with various configurations, your Ansible playbooks will become longer and more complex. You will likely need to re-use certain instructions for multiple (groups of) hosts, e.g. installing common software. 

Introducing “roles”. The code below prescribes a group of hosts that shall have a set of roles, and on which a corresponding set of tasks shall be executed (that code is not shown).
<b>Roles</b> are 

```yaml
---
- hosts: wordpress_hosts
  roles:
    - nginx
    - php
    - mysql
    - wordpress
```

In [ ]:
Running a playbook, basic syntax:

In [ ]:
ansible-playbook playbook.yml

# Ansible 

Docker, Ansible, EC2 References
=================

<https://www.digitalocean.com/community/tutorials/how-to-create-ansible-playbooks-to-automate-system-configuration-on-ubuntu>

<https://www.digitalocean.com/community/tutorials/how-to-use-ansible-roles-to-abstract-your-infrastructure-environment>

<https://dantehranian.wordpress.com/2015/01/20/ansible-vs-puppet-overview/>
Relatively recent (Jan 2015) comparison of tools

<http://www.ansible.com/blog/2013/12/08/the-origins-of-ansible>
The Origins of Ansible

<http://ops-school.readthedocs.org/en/latest/config_management.html>
A Brief History of Configuration Management


Video Tutorials
---------------
- sysadmincasts: Ansible
    <https://sysadmincasts.com/episodes/43-19-minutes-with-ansible-part-1-4>
    
- Comparison of configuration tools -- though somewhat dated (2013)
    <http://www.infoworld.com/article/2609482/data-center/data-center-review-puppet-vs-chef-vs-ansible-vs-salt.html>